In [4]:
import numpy as np
import pandas as pd
import matplotlib as plt
import scipy
# 不用print，直接显示结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 显示所有列
pd.set_option('display.max_columns', 600)

In [6]:
pd.__version__
np.__version__
plt.__version__
scipy.__version__

'1.0.3'

'1.18.1'

'3.1.3'

'1.4.1'

# 练习
## 【练习一】现有一份虚拟数据集，列类型分别为string/浮点/整型，请解决如下问题：
### （a）请以列类型读入数据，并选出C为缺失值的行。
### （b）现需要将A中的部分单元转为缺失值，单元格中的最小转换概率为25%，且概率大小与所在行B列单元的值成正比。

In [57]:
# a 请以列类型读入数据，并选出C为缺失值的行。
data = pd.read_csv('data/Missing_data_one.csv').convert_dtypes()
data.info()
data.head()

# C为缺失值的行
data[data['C'].isna()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       30 non-null     string 
 1   B       30 non-null     float64
 2   C       25 non-null     Int64  
dtypes: Int64(1), float64(1), string(1)
memory usage: 878.0 bytes


,A,B,C
0,not_NaN,0.922,4
1,not_NaN,0.700,<NA>
2,not_NaN,0.503,8
3,not_NaN,0.938,4
4,not_NaN,0.952,10


,A,B,C
1,not_NaN,0.700,<NA>
5,not_NaN,0.972,<NA>
11,not_NaN,0.736,<NA>
19,not_NaN,0.684,<NA>
21,not_NaN,0.913,<NA>


In [100]:
# （b）现需要将A中的部分单元转为缺失值，单元格中的最小转换概率为25%，且概率大小与所在行B列单元的值成正比。
data_convert = data.copy()

# 思路
## 按B的概率去转化成 nan, 其实这份数据里没有这种情况
## 如果B 的值小于25，就取25
## 总体思路就是 对A列，按 {值：概率 → [本身值 : 1-B，nan ：B]}去取值
### np.random.choice([0, 1, 2, 3], p = 概率列表) 该方法可以按p指定的概率，取前面list里的值
### data_convert['A'] = np.random.choice(['not_NaN',np.nan], p = [0.078,0.922])

# 解题
## B值小于0.25的话，就取0.25
data_convert.loc[data_convert['B']<0.25,'B'] = 0.25
## D为取缺失值的概率 
data_convert['D'] = 0.25 * data_convert['B']/ np.min(data_convert['B'])
## E为不取缺失值的概率
data_convert['E'] = 1 - 0.25 * data_convert['B']/ np.min(data_convert['B'])
#data_convert

## 逐行读取数据，用np.random.choice方法取值
## 将结果存入list中，最后将list的值替换原来的A列
Data_add = list()
for row in data_convert.itertuples():
    Data_add.append(np.random.choice(['not_NaN',None], p = [row[5],row[4]])) 

    
data['A'] = Data_add
data.head()

,A,B,C
0,None,0.922,4
1,not_NaN,0.700,<NA>
2,None,0.503,8
3,None,0.938,4
4,None,0.952,10


## 【练习二】 现有一份缺失的数据集，记录了36个人来自的地区、身高、体重、年龄和工资，请解决如下问题：
### （a）统计各列缺失的比例并选出在后三列中至少有两个非缺失值的行。
### （b）请结合身高列和地区列中的数据，对体重进行合理插值。

In [63]:
Missdata = pd.read_csv('data/Missing_data_two.csv').convert_dtypes()
Missdata.info()
Missdata.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   编号      36 non-null     Int64  
 1   地区      36 non-null     string 
 2   身高      36 non-null     float64
 3   体重      28 non-null     float64
 4   年龄      27 non-null     Int64  
 5   工资      28 non-null     Int64  
dtypes: Int64(3), float64(2), string(1)
memory usage: 1.9 KB


,编号,地区,身高,体重,年龄,工资
0,1,A,157.50,NaN,47,15905
1,2,B,202.00,91.80,25,<NA>
2,3,C,169.09,62.18,<NA>,<NA>
3,4,A,166.61,59.95,77,5434
4,5,B,185.19,NaN,62,4242


In [72]:
# （a）统计各列缺失的比例并选出在后三列中至少有两个非缺失值的行。
## 统计比例
Missdata.isnull().sum() / Missdata.shape[0]

## 筛选行缺失值个数
## df.isnull().sum(axis=1)
Missdata[Missdata[['体重','年龄','工资']].isnull().sum(axis=1) >= 2]

编号    0.000000
地区    0.000000
身高    0.000000
体重    0.222222
年龄    0.250000
工资    0.222222
dtype: float64

,编号,地区,身高,体重,年龄,工资
2,3,C,169.09,62.18,<NA>,<NA>
11,12,A,202.56,92.30,<NA>,<NA>
12,13,C,177.37,NaN,79,<NA>
14,15,C,199.11,89.20,<NA>,<NA>
26,27,B,158.28,NaN,51,<NA>
32,33,C,181.01,NaN,<NA>,13021
33,34,A,196.67,87.00,<NA>,<NA>


In [87]:
# （b）请结合身高列和地区列中的数据，对体重进行合理插值。

# 思路1
## 对身高分箱
## 按地区和分箱后的升高groupby，求体重的均值

# 解题
#cats1 = pd.cut(ages, bins)
bins = list(range(round(Missdata['身高'].min())-1,round(Missdata['身高'].max())+10,10))
Missdata['身高_bin'] = pd.cut(Missdata['身高'], bins)


0     (154, 164]
1     (194, 204]
2     (164, 174]
3     (164, 174]
4     (184, 194]
5     (184, 194]
6     (154, 164]
7     (174, 184]
8     (174, 184]
9     (184, 194]
10    (154, 164]
11    (194, 204]
12    (174, 184]
13    (174, 184]
14    (194, 204]
15    (164, 174]
16    (164, 174]
17    (184, 194]
18    (164, 174]
19    (154, 164]
20    (194, 204]
21    (154, 164]
22    (154, 164]
23    (164, 174]
24    (174, 184]
25    (164, 174]
26    (154, 164]
27    (164, 174]
28    (154, 164]
29    (174, 184]
30    (174, 184]
31    (164, 174]
32    (174, 184]
33    (194, 204]
34    (164, 174]
35    (174, 184]
Name: 身高, dtype: category
Categories (5, interval[int64]): [(154, 164] < (164, 174] < (174, 184] < (184, 194] < (194, 204]]

In [ ]:
# 思路2
## 根据身高和地区 多维插值
from scipy.interpolate import griddata
